### 필수 라이브러리 설치

In [ ]:
! pip install -q -U langchain-openai
! pip install -q -U langchain-community

### 라이브러리 임포트

In [ ]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_openai import ChatOpenAI
import pandas as pd

### 데이터 불러오기

In [ ]:
data = pd.read_csv("../data/train.csv")
display(data.head())

In [ ]:
import random

def create_examples_from_dataframe(df, num_examples):
    examples = []
    for _, row in df.sample(n=num_examples).iterrows():
        example = {
            "dialogue": row["dialogue"],
            "summary": row["summary"],
        }
        examples.append(example)
    return examples

In [ ]:
# 데이터프레임에서 examples 생성
examples = create_examples_from_dataframe(data, num_examples=len(data))
examples

In [ ]:
# 4개의 예시를 임의로 추출
few_shot_examples = random.sample(examples, 4)
few_shot_examples

### 프롬프트 템플릿

In [ ]:
# 예시 템플릿 정의
example_template = "[대화]\n{dialogue}\n\n[요약]\n{summary}"

# 예시 프롬프트 템플릿 생성
example_prompt = PromptTemplate(
    input_variables=["dialogue", "summary"],
    template=example_template
)

# Few-shot 프롬프트 템플릿 정의
few_shot_prompt = FewShotPromptTemplate(
    examples=few_shot_examples,  # 2개의 예시 사용
    example_prompt=example_prompt,
    prefix="당신은 대화를 요약해주는 유능한 AI입니다. \
당신의 임무는 다음에 나오는 대화를 요약하는 것입니다. \
당신의 대답은 오직 제공된 대화에만 근거해야 합니다. \
요약 스타일을 유지해야만 합니다.",
    suffix="[대화]\n{dialogue}\n\n[요약]\n ",
    input_variables=["dialogue"]
)

In [ ]:
test = pd.read_csv('../data/test.csv')
test.head()

In [ ]:
# 새로운 대화 데이터
new_dialogue = test['dialogue'][0]
# Few-shot 프롬프트 생성
prompt = few_shot_prompt.format(dialogue=new_dialogue)

print(prompt)

### OpenAI API를 활용하여 요약문 생성

In [ ]:
llm = ChatOpenAI(model_name='gpt-4-turbo', openai_api_key="", temperature=0.3, max_tokens=512)

In [ ]:
prompt = few_shot_prompt.format(dialogue=test['dialogue'][0])
summary = llm.invoke(prompt)
print(summary.content.strip())

In [ ]:
from tqdm.notebook import tqdm

summary_list = []
for dialouge in tqdm(test['dialogue']):
    # 임의로 4개의 few-shot 예제를 추출
    few_shot_examples = random.sample(examples, 4)

    # Few-shot 프롬프트 템플릿 정의
    few_shot_prompt = FewShotPromptTemplate(
    examples=few_shot_examples,  # 2개의 예시 사용
    example_prompt=example_prompt,
    prefix="당신은 대화를 요약해주는 유능한 AI입니다. \
당신의 임무는 다음에 나오는 대화를 요약하는 것입니다. \
당신의 대답은 오직 제공된 대화에만 근거해야 합니다. \
요약 스타일을 유지해야만 합니다.",
    suffix="[대화]\n{dialogue}\n\n[요약]\n ",
    input_variables=["dialogue"]
)
    # Few-shot 프롬프트 생성
    prompt = few_shot_prompt.format(dialogue=dialouge)

    # 언어 모델을 사용하여 요약 생성
    summary = llm.invoke(prompt)
    print("="*20, "[ 대화 ]", "="*20)
    print(dialouge)
    print("="*20, "[ 요약 ]", "="*20)
    print(summary.content.strip())
    print()

    summary_list.append(summary.content.strip())

### 생성된 요약문 저장

In [ ]:
test['summary'] = summary_list

In [ ]:
test[['fname', 'summary']].to_csv("submission_fewshot.csv", index=False)